In [26]:
import json
import numpy as np
import pandas as pd

import MeCab

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.manifold import TSNE

import plotly.express as px


In [27]:
stories_json = []
with open('./output_storiesv2.json', 'r') as f:
    stories_json = stories_json + json.loads(f.read())

print(len(stories_json))
print(stories_json[1].keys())

407
dict_keys(['index', 'title', 'broadcasting_date', 'abstracts_list'])


In [28]:
stories_df = pd.DataFrame(stories_json)
stories_df["abstract_joined"] = stories_df["abstracts_list"].str.join(" ")
stories_df.head()

,index,title,broadcasting_date,abstracts_list,abstract_joined
0,0325,春だ！映画だ！３時間アニメ祭り\r\n「映画ドラえもん のび太の新魔界大冒険～７人の魔法使い...,2013年3月15日,[ 何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいな...,何をやってもうまくいかないのび太は、「魔法（まほう）が使えたら便利で楽しくなるにちがいない...
1,0326,「ばくはつコショウ」「何が何でもお花見を」[2013年4月12日放送],2013年4月12日,[ ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。この...,ある朝、ねぼうしたのび太に泣きつかれたドラえもんは、『ばくはつコショウ』を取り出す。このコ...
2,0327,「やりクリしてハワイ旅行」「しずかちゃんのはごろも」[2013年4月26日放送],2013年4月26日,[ スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとする...,スネ夫からハワイ旅行をじまんされたのび太は、自分もハワイに行きたいとママにたのもうとするが...
3,0328,「ひるねは天国で」「どくさいスイッチ」[2013年5月3日放送],2013年5月3日,[ のび太とドラえもんが部屋で昼寝をしていると、突然大きな音が聞こえてくる。となりの家が改築...,のび太とドラえもんが部屋で昼寝をしていると、突然大きな音が聞こえてくる。となりの家が改築工...
4,0329,「インスタントママ」「狙われたジャイアン」[2013年5月10日放送],2013年5月10日,[ そうじにせんたく、町内会の会合と毎日大忙しのママが、「体がもうひとつほしい」とため息をつ...,そうじにせんたく、町内会の会合と毎日大忙しのママが、「体がもうひとつほしい」とため息をつい...


In [29]:
t = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
def mecab_tokenizer(text):
    parsed_lines = t.parse(text).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    # 原型を取得
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]

    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 瀕死を絞り込み
    target_pos = ["名詞"]
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]
    # アルファベットを小文字に統一
    token_list = [t.lower() for t in token_list]

    return [ text for text in filter(lambda x: not x.isascii(), token_list)]

In [30]:
# ストーリーが取得できたものに絞る
stories_df = stories_df[stories_df['abstract_joined'].str.len() > 0] 
sentences = stories_df["abstract_joined"].apply(mecab_tokenizer)
print(sentences)

0      [何, のび太, 魔法, 魔法, 便利, ちがい, もしもボックス, 現実, 世界, 魔法,...
1      [朝, 寝坊, のび太, ドラえもん, 爆発, コショウ, コショウ, 行き先, そば, 人...
2      [スネ夫, ハワイ旅行, じまん, のび太, 自分, ハワイ, ママ, 給料日, 前, お金...
3      [のび太, ドラえもん, 部屋, 昼寝, 音, となり, 家, 改築, 工事, の, しずか...
4      [そうじ, せんたく, 町内会, 会合, 毎日, 大忙し, ママ, 体, ひとつ, ため息,...
                             ...                        
402    [しずか, 遊び, ジャイアン, 意味, ボコボコ, のび太, 家, 部屋, 中, ドラえも...
403    [スネ夫, 手, 恐竜, きょう, りゅう, プラモデル, じまん, のび太, プラモデル,...
404    [秋, ため, 別荘, べっそう, スネ夫, 季節感, 顔, バカ, のび太, 今年, ふり...
405    [空き地, サッカー, ジャイアン, スネ夫, たち, ジャイアン, ボール, 神成, さん...
406    [使い, の, ちゅ, 腕木, げん, そう, ジャイアン, 別, 道, のび太, 空き, ...
Name: abstract_joined, Length: 407, dtype: object


In [31]:
abstract_list = sentences.tolist()
trainings = [TaggedDocument(words = data, tags = [i]) for i,data in enumerate(abstract_list)]
m = Doc2Vec(documents= trainings, dm = 1, window=8, min_count=10, workers=4)


In [32]:
vector_list = m.dv.vectors
tsne = TSNE(n_components=2)
vector_enbedded = tsne.fit_transform(vector_list)
print(vector_enbedded)

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:795: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/Users/takamirei/.pyenv/versions/3.9.5/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:805: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[[ 1.72701950e+01  5.95560169e+00]
 [-7.44685936e+00 -6.86362013e-02]
 [ 9.87161160e+00  6.37883425e+00]
 [ 4.03299189e+00  1.35644639e+00]
 [-9.92776966e+00 -2.67056298e+00]
 [ 1.33034773e+01 -2.73063630e-01]
 [ 3.15951538e+01  8.11937523e+00]
 [ 2.95219631e+01  7.47636938e+00]
 [ 2.79069691e+01  8.06144619e+00]
 [ 1.52776594e+01  2.68543673e+00]
 [ 6.37369990e-01 -4.11910391e+00]
 [-6.00746441e+00  1.62159908e+00]
 [-3.03506298e+01 -1.54586716e+01]
 [-7.92027378e+00  2.64707971e+00]
 [-1.45414896e+01 -2.62022233e+00]
 [-5.00126600e+00  3.72334170e+00]
 [-1.28849754e+01 -9.91637528e-01]
 [ 9.21696186e+00 -9.02975440e-01]
 [-1.81149044e+01 -9.61924267e+00]
 [ 1.42526376e+00 -1.24116075e+00]
 [ 1.19978113e+01 -8.61009181e-01]
 [ 2.56128998e+01  8.00540161e+00]
 [ 2.83148899e+01  9.44601440e+00]
 [ 2.16847630e+01  6.87782335e+00]
 [ 1.81477757e+01  8.62890816e+00]
 [-8.21162701e+00 -1.87057328e+00]
 [ 1.17431993e+01  2.89807940e+00]
 [-3.70070863e+00 -1.83676016e+00]
 [ 3.51674271e+01  8

In [33]:
vector_x = [v[0] for  v in vector_enbedded]
vector_y = [v[1] for  v in vector_enbedded]

fig = px.scatter(x=vector_x, y=vector_y)
fig.show()